<p align="center">
  <img src="https://raw.githubusercontent.com/Duelion/whatsapp-wrapped/main/.github/assets/ws_logo.png" alt="WhatsApp Wrapped Logo" width="401">
</p>

<p align="center">
  <strong>Create beautiful Spotify Wrapped-style visualizations for your WhatsApp group chats!</strong>
</p>

<p align="center">
  <a href="https://duelion.github.io/whatsapp-wrapped/sample_report.html">
    <img src="https://img.shields.io/badge/🔍_View_Sample_Report-4CAF50?style=for-the-badge" alt="View Sample Report">
  </a>
</p>

<div align="center">

| | |
|:---:|:---:|
| 📈 **Rich Analytics** — message counts, patterns & emoji stats | 🎨 **Interactive Charts** — beautiful Plotly visualizations |
| 📅 **Calendar Heatmaps** — year-at-a-glance activity | 🔒 **100% Private** — all processing stays in this notebook |

</div>

---

## 🚀 How to Use This Notebook

**New to Google Colab?** No worries! This is like a document with runnable code cells. Each "Step" below is a code cell — click on it and press the **▶ Play button** on the left (or press `Ctrl+Enter` / `Cmd+Enter`) to run it. Run them in order from top to bottom!

1. **Run Step 1** - Install dependencies and WebKit (only needed once per session)
2. **Run Step 2** - Upload your chat file and see available options
3. **Run Step 3** - Configure filters and generate your report!

💡 **Tip:** If you see "Runtime disconnected" or come back after a break, start from Step 1 again.

---

<details>
<summary>📱 <b>How to Export WhatsApp Chat</b></summary>

1. Open WhatsApp and navigate to the group chat
2. Tap the group name → More → Export chat
3. Choose **"Without Media"** for faster processing
4. Save the `.zip` file to upload here

</details>

🔒 **Privacy First**: All processing happens in this notebook. Your data is never uploaded anywhere else.

In [ ]:
#@title 🔧 **Step 1: Setup** { display-mode: "form" }
#@markdown This cell installs all required dependencies, WebKit browser, and clones the WhatsApp Wrapped repository.

import os
import subprocess
import sys
import threading
import time
from pathlib import Path

from IPython.display import HTML, clear_output, display


def run_with_spinner(cmd, message):
    """Run a subprocess command with an animated spinner."""
    spinner_chars = "⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏"
    stop_spinner = threading.Event()
    result_holder = [None]

    def spin():
        i = 0
        while not stop_spinner.is_set():
            print(f"\r  • {message} {spinner_chars[i % len(spinner_chars)]}", end="", flush=True)
            time.sleep(0.1)
            i += 1

    def run_cmd():
        result_holder[0] = subprocess.run(cmd, capture_output=True, text=True)
        stop_spinner.set()

    # Start spinner and command
    spinner_thread = threading.Thread(target=spin)
    cmd_thread = threading.Thread(target=run_cmd)

    spinner_thread.start()
    cmd_thread.start()

    cmd_thread.join()
    spinner_thread.join()

    # Clear the spinner line and print result
    result = result_holder[0]
    if result.returncode == 0:
        print(f"\r  • {message} ✓" + " " * 10)
    else:
        print(f"\r  • {message} ⚠️" + " " * 10)

    return result

def install_packages():
    """Install all required packages."""
    print("📦 Installing dependencies...")
    print("="*50)

    # Install all packages
    packages = [
        "polars>=1.0.0",
        "stop-words>=2018.7.23",
        "matplotlib>=3.7.0",
        "seaborn>=0.12.0",
        "plotly>=5.14.0",
        "plotly-calplot>=0.1.20",
        "wordcloud>=1.9.0",
        "pillow>=10.0.0",
        "python-dateutil>=2.8.0",
        "pyyaml>=6.0",
        "tqdm>=4.65.0",
        "jinja2>=3.1.0",
        "emojis>=0.7.0",
        "playwright",
        "nest_asyncio",
    ]

    for pkg in packages:
        pkg_name = pkg.split(">=")[0].split("[")[0]
        run_with_spinner(
            [sys.executable, "-m", "pip", "install", "-q", pkg],
            f"Installing {pkg_name}..."
        )

    print("\n" + "="*50)
    print("✅ Packages installed!")
    print("="*50)

def install_webkit():
    """Install WebKit browser for static HTML generation."""
    marker = "/content/.playwright_installed"

    if os.path.exists(marker):
        print("\n🌐 WebKit already installed!")
        print("="*50)
        return

    print("\n🌐 Installing WebKit browser...")
    print("="*50)
    print("   (This enables Static HTML generation for WhatsApp sharing)")
    print("")

    # Install system dependencies
    run_with_spinner(
        [sys.executable, "-m", "playwright", "install-deps", "webkit"],
        "Installing system dependencies..."
    )

    # Install WebKit
    result = run_with_spinner(
        [sys.executable, "-m", "playwright", "install", "webkit"],
        "Installing WebKit browser (~80MB)..."
    )
    if result.returncode == 0:
        with open(marker, "w") as f:
            f.write("done")

    print("\n" + "="*50)
    print("✅ WebKit installed!")
    print("="*50)

def clone_repository():
    """Clone or update the WhatsApp Wrapped repository."""
    print("\n📥 Setting up WhatsApp Wrapped repository...")
    print("="*50)

    repo_path = "/content/whatsapp-wrapped"

    if os.path.exists(repo_path):
        print("  • Repository already exists, updating...")
        result = subprocess.run(
            ["git", "-C", repo_path, "pull", "--quiet"],
            capture_output=True, text=True
        )
        print("  ✓ Repository updated!")
    else:
        result = subprocess.run(
            ["git", "clone", "--depth", "1",
             "https://github.com/Duelion/whatsapp-wrapped.git", repo_path],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            print("  ✓ Repository cloned successfully!")
        else:
            print(f"  ⚠️ Clone failed: {result.stderr}")

    # Add to Python path
    if repo_path not in sys.path:
        sys.path.insert(0, repo_path)

    return repo_path

# ============================================================================
# MAIN SETUP LOGIC
# ============================================================================

# Install packages
install_packages()

# Clone/update repository
repo_path = clone_repository()

# Install WebKit for static HTML generation
install_webkit()

# Clear the verbose progress output and show clean success message
clear_output()

# Display styled success message
display(HTML("""
<div style="background: linear-gradient(135deg, #1DB954, #191414);
            padding: 20px; border-radius: 12px; margin-top: 20px; max-width: 720px;
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);">
    <h3 style="color: white; margin: 0;">🎉 Ready to go!</h3>
    <p style="color: #e8e8e8; margin: 10px 0 0 0; font-size: 15px;">All dependencies installed and WebKit is ready for Static HTML generation.</p>
    <p style="color: #b3b3b3; margin: 10px 0 0 0; font-size: 13px;">You can now proceed to <strong>Step 2</strong> to upload your chat file.</p>
</div>
"""))

In [ ]:
#@title 📤 **Step 2: Upload Chat File** { display-mode: "form" }
#@markdown Upload your WhatsApp chat export (.zip or .txt) to analyze it.
#@markdown
#@markdown After uploading, you'll see available years and options for your chat.

import json

import ipywidgets as widgets
import polars as pl
from google.colab import files
from IPython.display import HTML, clear_output, display

# Check if setup was completed
repo_path = "/content/whatsapp-wrapped"
if not os.path.exists(repo_path):
    display(HTML("""
    <div style="background: linear-gradient(135deg, #d32f2f, #191414); padding: 20px; border-radius: 12px; max-width: 720px;
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                border: 1px solid rgba(211, 47, 47, 0.15);">
        <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ Setup not completed</h4>
        <p style="color: #ffcdd2; margin: 0;">Please run <strong>Step 1</strong> first to install dependencies and clone the repository.</p>
    </div>
    """))
else:
    # Display output format info header
    display(HTML("""
    <div style="background: linear-gradient(145deg, #1e1e1e 0%, #2d2d2d 100%); padding: 15px; border-radius: 12px; margin-bottom: 20px; max-width: 720px;
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                border: 1px solid rgba(255, 255, 255, 0.05);">
        <h4 style="color: #1DB954; margin: 0 0 10px 0;">📄 Output: Static HTML (Mobile) + Interactive HTML (Desktop)</h4>
        <p style="color: #e8e8e8; margin: 0; font-size: 14px;">
            <strong>📱 Static HTML (Mobile)</strong> — Perfect for sharing on WhatsApp or opening on phone<br>
            <strong>💻 Interactive HTML (Desktop)</strong> — Full experience with hover tooltips, zoom & pan on PC
        </p>
    </div>
    """))

    print("📤 Please upload your WhatsApp chat export (.zip or .txt)...")
    print("="*60)

    try:
        uploaded = files.upload()
    except Exception as e:
        print(f"\n⚠️ Upload cancelled or failed: {e}")
        uploaded = None

    if uploaded:
        # Get the uploaded filename
        filename = list(uploaded.keys())[0]
        file_content = uploaded[filename]

        print(f"\n✓ Uploaded: {filename} ({len(file_content):,} bytes)")
        print("\n" + "="*60)
        print("🔍 Analyzing chat structure...")
        print("="*60 + "\n")

        # Save the uploaded file to a persistent location
        upload_dir = "/content/uploaded_chat"
        os.makedirs(upload_dir, exist_ok=True)
        chat_path = os.path.join(upload_dir, filename)
        with open(chat_path, 'wb') as f:
            f.write(file_content)

        try:
            # Ensure repo path is in sys.path before importing
            repo_path = "/content/whatsapp-wrapped"
            if repo_path not in sys.path:
                sys.path.insert(0, repo_path)

            from whatsapp_wrapped.parser import parse_whatsapp_export

            # Quick parse to extract metadata (no filtering)
            print("[1/2] 📖 Parsing chat file...")
            df_full, metadata_full = parse_whatsapp_export(
                chat_path,
                filter_system=True,
                min_messages=1,  # Include all users for analysis
                year_filter=None,  # No year filter for initial analysis
            )
            print(f"      ✓ Found {len(df_full):,} messages from {metadata_full.total_members} members")

            # Extract available years
            print("[2/2] 📅 Extracting available years...")
            available_years = sorted(df_full['timestamp'].dt.year().unique().to_list(), reverse=True)
            years_with_counts_df = df_full.group_by(pl.col('timestamp').dt.year().alias('year')).len().sort('year', descending=True)
            years_with_counts = dict(zip(years_with_counts_df['year'].to_list(), years_with_counts_df['len'].to_list(), strict=False))
            print(f"      ✓ Found messages from {len(available_years)} years")

            # Store data for next cell
            # Using a simple approach: save to files that the next cell can read
            analysis_info = {
                "chat_path": chat_path,
                "filename": filename,
                "total_messages": len(df_full),
                "total_members": metadata_full.total_members,
                "available_years": [int(y) for y in available_years],
                "years_with_counts": {str(k): int(v) for k, v in years_with_counts.items()},
                "date_range_start": metadata_full.date_range_start.isoformat(),
                "date_range_end": metadata_full.date_range_end.isoformat(),
                "member_names": metadata_full.member_names,
            }
            with open("/content/chat_analysis.json", "w") as f:
                json.dump(analysis_info, f)

            # Clear the verbose progress output and show clean results
            clear_output()

            # Check for unsaved contacts (names with ~, phone numbers, or empty names)
            import re
            def is_phone_number(name):
                """Check if a name looks like a phone number (mostly digits, spaces, +, -)."""
                if not name:
                    return False
                # Remove common phone number characters and check if mostly digits remain
                cleaned = re.sub(r'[\s\-\+\(\)]', '', name)
                # If it's mostly digits (at least 6 and 80%+ are digits), it's likely a phone number
                if len(cleaned) >= 6:
                    digit_count = sum(c.isdigit() for c in cleaned)
                    return digit_count / len(cleaned) >= 0.8
                return False

            unsaved_contacts = [name for name in metadata_full.member_names
                               if not name or '~' in name or name.strip() == '' or is_phone_number(name)]
            if unsaved_contacts:
                unsaved_list = ', '.join([f'"{n}"' if n else '(empty)' for n in unsaved_contacts[:5]])
                if len(unsaved_contacts) > 5:
                    unsaved_list += f', and {len(unsaved_contacts) - 5} more...'

                display(HTML(f"""
                <div style="background: linear-gradient(135deg, #f57f17, #191414); padding: 20px; border-radius: 12px; margin: 20px 0; max-width: 720px;
                            border-left: 4px solid #ffeb3b;
                            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);">
                    <h4 style="color: #ffeb3b; margin: 0 0 12px 0;">📱 Tip: Some contacts are not saved</h4>
                    <p style="color: #fff9c4; margin: 0 0 12px 0; font-size: 14px;">
                        We detected <strong>{len(unsaved_contacts)}</strong> member(s) showing as phone numbers or empty names:
                    </p>
                    <p style="color: #ffd54f; margin: 0 0 12px 0; font-size: 13px; font-family: monospace;">
                        {unsaved_list}
                    </p>
                    <p style="color: #fff9c4; margin: 0; font-size: 14px;">
                        <strong>For better results:</strong> Add these contacts to your phone, then export the chat again.
                        Names like "~+1234567890" will be replaced with the saved contact name.
                    </p>
                </div>
                """))

            # Build year options HTML
            year_rows = ""
            for year, count in years_with_counts.items():
                pct = (count / len(df_full)) * 100
                year_rows += f"""
                    <tr>
                        <td style="padding: 6px 16px 6px 0; color: #1DB954; font-weight: 600;">{year}</td>
                        <td style="padding: 6px 16px 6px 0; color: #e8e8e8;">{count:,} messages</td>
                        <td style="padding: 6px 0; color: #888;">({pct:.1f}%)</td>
                    </tr>"""

            # Display analysis results
            display(HTML(f"""
            <div style="background: linear-gradient(145deg, #1e1e1e 0%, #2d2d2d 100%);
                        padding: 20px; border-radius: 12px; margin: 20px 0; max-width: 720px;
                        border-left: 3px solid #1DB954;
                        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                        box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                        border: 1px solid rgba(255, 255, 255, 0.05);">
                <div style="color: #e8e8e8; font-size: 16px; font-weight: 600; margin-bottom: 12px;">
                    📊 {metadata_full.filename}
                </div>
                <div style="display: flex; gap: 20px; margin: 12px 0; color: #e8e8e8; font-size: 14px;">
                    <span><strong style="color: #1DB954;">{len(df_full):,}</strong> messages</span>
                    <span>•</span>
                    <span><strong style="color: #1DB954;">{metadata_full.total_members}</strong> members</span>
                    <span>•</span>
                    <span><strong style="color: #1DB954;">{len(available_years)}</strong> years</span>
                </div>
                <div style="color: #b3b3b3; font-size: 13px; margin-top: 8px;">
                    {metadata_full.date_range_start.strftime('%b %d, %Y')} - {metadata_full.date_range_end.strftime('%b %d, %Y')}
                </div>
            </div>

            <div style="background: linear-gradient(145deg, #1e1e1e 0%, #2d2d2d 100%); padding: 18px; border-radius: 12px; margin-top: 10px; max-width: 720px;
                        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                        box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                        border: 1px solid rgba(255, 255, 255, 0.05);">
                <div style="color: #1DB954; font-size: 13px; font-weight: 600; margin-bottom: 10px; text-transform: uppercase; letter-spacing: 0.5px;">
                    📅 Available Years
                </div>
                <table style="color: #e8e8e8; border-collapse: collapse; width: 100%; font-size: 14px;">
                    {year_rows}
                </table>
            </div>

            <div style="background: linear-gradient(135deg, #1DB954, #191414);
                        padding: 20px; border-radius: 12px; margin-top: 20px; max-width: 720px;
                        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                        box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);">
                <h3 style="color: white; margin: 0 0 10px 0;">✓ Ready to generate!</h3>
                <p style="color: #e8e8e8; margin: 0; font-size: 15px;">
                    👉 Proceed to <strong>Step 3</strong> to configure and generate your report, or upload a different file below.
                </p>
            </div>
            """))

            # Add button to upload a different file
            reupload_output = widgets.Output()

            reupload_btn = widgets.Button(
                description='📤 Upload Different File',
                button_style='',
                layout=widgets.Layout(width='200px', height='40px')
            )

            def on_reupload(btn):
                # Remove the analysis file to allow re-upload
                if os.path.exists("/content/chat_analysis.json"):
                    os.remove("/content/chat_analysis.json")
                with reupload_output:
                    clear_output()
                    display(HTML("""
                    <div style="background:linear-gradient(135deg,#ff9800,#191414);
                                padding:20px;border-radius:12px;max-width:720px;
                                box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                                border: 1px solid rgba(255, 152, 0, 0.15);">
                      <h3 style="color:white;margin:0 0 10px 0;">🔄 Re-run this cell to upload a new file</h3>
                      <p style="color:#e8e8e8;margin:0;">Click the <strong>Run</strong> button on this cell or press <strong>Ctrl+Enter</strong> to upload a different chat file.</p>
                    </div>
                    """))

            reupload_btn.on_click(on_reupload)

            display(HTML("""
            <div style="margin-top: 15px;">
            </div>
            """))
            display(reupload_btn)
            display(reupload_output)

            # Memory cleanup: delete large dataframe after analysis
            import gc
            del df_full
            gc.collect()

        except Exception as e:
            import traceback
            print(f"\n❌ Error analyzing chat: {e}")
            print("\nFull error details:")
            traceback.print_exc()

            display(HTML("""
            <div style="background: linear-gradient(135deg, #d32f2f, #191414); padding: 20px; border-radius: 12px; margin-top: 20px; max-width: 720px;
                        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                        box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                        border: 1px solid rgba(211, 47, 47, 0.15);">
                <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ Troubleshooting Tips</h4>
                <ul style="color: #ffcdd2; margin: 0; padding-left: 20px;">
                    <li>Make sure you uploaded a valid WhatsApp export file (.zip or .txt)</li>
                    <li>The file should be exported "Without Media" from WhatsApp</li>
                    <li>Try running Step 1 (Setup) again if you see import errors</li>
                </ul>
            </div>
            """))
    else:
        display(HTML("""
        <div style="background: linear-gradient(135deg, #f57f17, #191414); padding: 20px; border-radius: 12px; margin-top: 20px; max-width: 720px;
                    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                    box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                    border: 1px solid rgba(245, 127, 23, 0.15);">
            <h4 style="color: #ffeb3b; margin: 0 0 10px 0;">📤 No file uploaded</h4>
            <p style="color: #fff9c4; margin: 0;">Run this cell again and upload your WhatsApp chat export file.</p>
        </div>
        """))

In [ ]:
#@title 🎛️ **Step 3: Configure & Generate Report** { display-mode: "form" }
#@markdown Configure your report options and generate!
#@markdown
#@markdown Select which users and years to include in your report.

import json
import os
import re
import sys
from datetime import datetime

import ipywidgets as widgets
import polars as pl
from google.colab import files
from IPython.display import HTML, clear_output, display


def run_task_with_spinner(task_func, message):
    """Run a Python function with a simple status indicator.

    Note: Animated spinners with \\r don't work inside widgets.Output() in Colab,
    so we use a simple approach: show message, run task, show result.
    """
    result_holder = [None]
    error_holder = [None]

    # Show initial message with hourglass
    print(f"  • {message} ⏳", end="", flush=True)

    # Run the task synchronously
    try:
        result_holder[0] = task_func()
    except Exception as e:
        error_holder[0] = e

    # Clear line and show result
    if error_holder[0]:
        print(f"\r  • {message} ❌")
        raise error_holder[0]
    else:
        print(f"\r  • {message} ✓")

    return result_holder[0]

def sanitize_filename(name):
    """
    Sanitize a string to be safe for use as a filename on Windows/Mac/Linux.
    Keeps it human-readable while removing problematic characters.
    """
    if not name:
        return "report"

    # Remove characters that are invalid on Windows/Mac: < > : " / \ | ? *
    # Also remove control characters (0-31)
    invalid_chars = r'[<>:"/\\|?*\x00-\x1f]'
    name = re.sub(invalid_chars, '', name)

    # Replace multiple spaces with single space
    name = re.sub(r'\s+', ' ', name)

    # Strip leading/trailing whitespace and dots (Windows doesn't like trailing dots)
    name = name.strip(' .')

    # If the name is empty after sanitization, use a default
    if not name:
        return "report"

    return name

# Ensure repo is in sys.path
repo_path = "/content/whatsapp-wrapped"
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

# Check if setup was completed
if not os.path.exists(repo_path):
    display(HTML("""
    <div style="background: linear-gradient(135deg, #d32f2f, #191414); padding: 20px; border-radius: 12px; max-width: 720px; width: 100%; box-sizing: border-box;
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                border: 1px solid rgba(211, 47, 47, 0.15);">
        <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ Setup not completed</h4>
        <p style="color: #ffcdd2; margin: 0;">Please run <strong>Step 1</strong> first to install dependencies and clone the repository.</p>
    </div>
    """))
elif not os.path.exists("/content/chat_analysis.json"):
    display(HTML("""
    <div style="background: linear-gradient(135deg, #d32f2f, #191414); padding: 20px; border-radius: 12px; max-width: 720px; width: 100%; box-sizing: border-box;
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                border: 1px solid rgba(211, 47, 47, 0.15);">
        <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ No chat uploaded</h4>
        <p style="color: #ffcdd2; margin: 0;">Please run <strong>Step 2</strong> first to upload your WhatsApp chat file.</p>
    </div>
    """))
else:
    # Load analysis data
    with open("/content/chat_analysis.json") as f:
        analysis = json.load(f)

    # Always generate both HTML types (no format selection needed)
    generate_static_html = True

    # Build year options from actual chat data (only specific years, no "All Years")
    year_options = [str(y) for y in analysis["available_years"]]

    # Build user options with message counts
    member_names = analysis["member_names"]

    # Load chat data to get message counts per user and per year
    from whatsapp_wrapped.parser import parse_whatsapp_export
    df_temp, _ = parse_whatsapp_export(
        analysis["chat_path"],
        filter_system=True,
        min_messages=1,
        year_filter=None,
    )

    # Calculate message counts per user overall and per year
    user_counts_df = df_temp.group_by("name").len()
    user_message_counts = dict(zip(user_counts_df['name'].to_list(), user_counts_df['len'].to_list(), strict=False))

    # Calculate per-year user message counts
    df_temp = df_temp.with_columns(pl.col('timestamp').dt.year().alias('year'))
    user_message_counts_by_year = {}
    for year in analysis["available_years"]:
        year_df = df_temp.filter(pl.col('year') == year)
        year_counts_df = year_df.group_by("name").len()
        user_message_counts_by_year[year] = dict(zip(year_counts_df['name'].to_list(), year_counts_df['len'].to_list(), strict=False))

    # Memory cleanup: delete temp dataframe after extracting counts
    import gc
    del df_temp
    gc.collect()

    # Inject custom CSS for beautiful styling
    display(HTML("""
    <style>
        /* Container styling */
        .wrapped-config-card {
            background: linear-gradient(145deg, #1e1e1e 0%, #2d2d2d 100%);
            border-radius: 12px;
            padding: 24px;
            margin: 16px 0;
            max-width: 720px;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
            border: 1px solid rgba(255, 255, 255, 0.05);
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
        }

        .wrapped-header {
            background: linear-gradient(135deg, #1DB954, #191414);
            border-radius: 12px;
            padding: 20px;
            margin-bottom: 24px;
            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
        }

        .wrapped-header h3 {
            color: white;
            margin: 0 0 8px 0;
            font-size: 18px;
            font-weight: 700;
            display: flex;
            align-items: center;
            gap: 10px;
        }

        .wrapped-header .stats {
            color: rgba(255, 255, 255, 0.9);
            font-size: 14px;
            display: flex;
            gap: 16px;
            flex-wrap: wrap;
        }

        .wrapped-header .stat-item {
            background: rgba(0, 0, 0, 0.2);
            padding: 4px 12px;
            border-radius: 20px;
            font-size: 13px;
        }

        .wrapped-section {
            background: rgba(255, 255, 255, 0.03);
            border-radius: 12px;
            padding: 20px;
            margin-bottom: 16px;
            border: 1px solid rgba(255, 255, 255, 0.05);
        }

        .wrapped-section-title {
            color: #1DB954;
            font-size: 13px;
            font-weight: 600;
            text-transform: uppercase;
            letter-spacing: 1px;
            margin-bottom: 16px;
            display: flex;
            align-items: center;
            gap: 8px;
        }

        .wrapped-section-title::after {
            content: '';
            flex: 1;
            height: 1px;
            background: linear-gradient(90deg, rgba(29, 185, 84, 0.3) 0%, transparent 100%);
        }

        /* Custom dropdown styling */
        .widget-dropdown select {
            background: #3a3a3a !important;
            border: 2px solid #4a4a4a !important;
            border-radius: 8px !important;
            color: white !important;
            padding: 10px 16px !important;
            font-size: 14px !important;
            cursor: pointer !important;
            transition: all 0.2s ease !important;
        }

        .widget-dropdown select:hover {
            border-color: #1DB954 !important;
            box-shadow: 0 0 0 3px rgba(29, 185, 84, 0.15) !important;
        }

        .widget-dropdown select:focus {
            border-color: #1DB954 !important;
            outline: none !important;
            box-shadow: 0 0 0 3px rgba(29, 185, 84, 0.25) !important;
        }

        /* Button styling */
        .wrapped-btn-group {
            display: flex;
            gap: 8px;
            margin-bottom: 12px;
        }

        .widget-button button {
            border-radius: 8px !important;
            font-weight: 600 !important;
            text-transform: uppercase !important;
            font-size: 11px !important;
            letter-spacing: 0.5px !important;
            transition: all 0.2s ease !important;
            border: none !important;
        }

        .widget-button button:hover {
            transform: translateY(-1px) !important;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.3) !important;
        }

        /* Checkbox container styling */
        .wrapped-checkbox-container {
            background: linear-gradient(145deg, #1e1e1e 0%, #2d2d2d 100%) !important;
            border: 1px solid rgba(255, 255, 255, 0.05) !important;
            border-radius: 12px !important;
            padding: 8px !important;
            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3) !important;
        }

        .widget-checkbox {
            transition: all 0.15s ease !important;
            padding: 6px 8px !important;
            border-radius: 6px !important;
            margin: 2px 0 !important;
        }

        .widget-checkbox:hover {
            background: rgba(29, 185, 84, 0.1) !important;
        }

        .widget-checkbox input[type="checkbox"] {
            width: 18px !important;
            height: 18px !important;
            accent-color: #1DB954 !important;
            cursor: pointer !important;
        }

        .widget-checkbox label {
            color: #e8e8e8 !important;
            font-size: 13px !important;
            cursor: pointer !important;
        }

        /* Text input styling */
        .widget-text input {
            background: #3a3a3a !important;
            border: 2px solid #4a4a4a !important;
            border-radius: 8px !important;
            color: white !important;
            padding: 10px 16px !important;
            font-size: 14px !important;
            transition: all 0.2s ease !important;
        }

        .widget-text input:hover {
            border-color: #5a5a5a !important;
        }

        .widget-text input:focus {
            border-color: #1DB954 !important;
            outline: none !important;
            box-shadow: 0 0 0 3px rgba(29, 185, 84, 0.25) !important;
        }

        .widget-text input::placeholder {
            color: #888 !important;
        }

        /* Custom button styles matching report aesthetic */
        .wrapped-btn-select button {
            background: linear-gradient(135deg, #3f3f46 0%, #27272a 100%) !important;
            color: #fafafa !important;
            border: 1px solid #52525b !important;
        }

        .wrapped-btn-select button:hover {
            background: linear-gradient(135deg, #52525b 0%, #3f3f46 100%) !important;
            border-color: #71717a !important;
        }

        .wrapped-btn-clear button {
            background: linear-gradient(135deg, #3f3f46 0%, #27272a 100%) !important;
            color: #f87171 !important;
            border: 1px solid #52525b !important;
        }

        .wrapped-btn-clear button:hover {
            background: linear-gradient(135deg, #52525b 0%, #3f3f46 100%) !important;
            border-color: #f87171 !important;
        }

        /* Download buttons matching report aesthetic */
        .wrapped-btn-download-html button {
            background: linear-gradient(135deg, #1DB954 0%, #1ed760 100%) !important;
            color: white !important;
            border: none !important;
            box-shadow: 0 4px 12px rgba(29, 185, 84, 0.3) !important;
        }

        .wrapped-btn-download-html button:hover {
            box-shadow: 0 6px 20px rgba(29, 185, 84, 0.4) !important;
        }

        .wrapped-btn-download-static button {
            background: linear-gradient(135deg, #ff9800 0%, #ffb74d 100%) !important;
            color: white !important;
            border: none !important;
            box-shadow: 0 4px 12px rgba(255, 152, 0, 0.3) !important;
        }

        .wrapped-btn-download-static button:hover {
            box-shadow: 0 6px 20px rgba(255, 152, 0, 0.4) !important;
        }

        /* Generate button styling */
        .wrapped-generate-btn button {
            background: linear-gradient(135deg, #1DB954 0%, #1ed760 100%) !important;
            border: none !important;
            border-radius: 12px !important;
            color: white !important;
            font-size: 15px !important;
            font-weight: 700 !important;
            padding: 14px 32px !important;
            cursor: pointer !important;
            transition: all 0.3s ease !important;
            box-shadow: 0 4px 20px rgba(29, 185, 84, 0.4) !important;
            text-transform: none !important;
            letter-spacing: normal !important;
        }

        .wrapped-generate-btn button:hover {
            transform: translateY(-2px) scale(1.02) !important;
            box-shadow: 0 8px 30px rgba(29, 185, 84, 0.5) !important;
        }

        .wrapped-generate-btn button:active {
            transform: translateY(0) scale(0.98) !important;
        }

        /* Label styling */
        .widget-label {
            color: #888 !important;
        }

        /* Mobile Responsive Styles */
        @media (max-width: 600px) {
            /* Make containers full width with proper padding */
            .wrapped-config-card,
            .wrapped-header,
            .wrapped-section {
                max-width: 100% !important;
                padding: 16px !important;
                margin-left: 0 !important;
                margin-right: 0 !important;
            }

            /* Force widgets to be full width */
            .widget-dropdown select,
            .widget-text input {
                width: 100% !important;
                max-width: 100% !important;
            }

            /* Make button groups wrap */
            .wrapped-btn-group {
                flex-wrap: wrap !important;
            }

            /* Stack stats grids vertically */
            .wrapped-header .stats {
                flex-direction: column !important;
                gap: 8px !important;
            }

            /* Make stat grids single column */
            div[style*="grid-template-columns: repeat(3, 1fr)"] {
                grid-template-columns: 1fr !important;
                gap: 10px !important;
            }

            /* Checkbox container adjustments */
            .wrapped-checkbox-container {
                max-width: 100% !important;
            }

            .widget-checkbox {
                max-width: 100% !important;
            }

            /* HBox layouts should wrap */
            .widget-hbox {
                flex-wrap: wrap !important;
            }

            /* Reduce font sizes slightly on mobile */
            .wrapped-header h3 {
                font-size: 16px !important;
            }

            .wrapped-section-title {
                font-size: 12px !important;
            }
        }
    </style>
    """))

    # Create widgets with improved styling
    year_dropdown = widgets.Dropdown(
        options=year_options,
        value=year_options[0],
        layout=widgets.Layout(width='auto', min_width='200px')
    )

    # Sort by dropdown for users
    sort_by_dropdown = widgets.Dropdown(
        options=[('Name', 'name'), ('Message Count', 'messages')],
        value='name',
        layout=widgets.Layout(width='auto', min_width='160px')
    )

    # User checkboxes (all selected by default, created for all users)
    user_checkboxes_dict = {}
    for name in member_names:
        msg_count = user_message_counts.get(name, 0)
        checkbox = widgets.Checkbox(
            value=True,
            description=f"{name} ({msg_count:,})",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='auto', min_width='280px')
        )
        user_checkboxes_dict[name] = checkbox

    # Function to get users for a specific year
    def get_users_for_year(year):
        """Get list of users who have messages in the given year."""
        year_int = int(year)
        year_counts = user_message_counts_by_year.get(year_int, {})
        return [name for name in member_names if name in year_counts]

    # Function to update checkbox descriptions based on year
    def update_checkbox_descriptions(year):
        """Update checkbox descriptions with year-specific message counts."""
        year_int = int(year)
        year_counts = user_message_counts_by_year.get(year_int, {})
        for name, checkbox in user_checkboxes_dict.items():
            count = year_counts.get(name, 0)
            checkbox.description = f"{name} ({count:,})"

    # Function to get sorted checkboxes filtered by year
    def get_sorted_checkboxes(sort_by, year):
        """Get checkboxes for users in the selected year, sorted by criteria."""
        users_in_year = get_users_for_year(year)
        year_int = int(year)
        year_counts = user_message_counts_by_year.get(year_int, {})

        if sort_by == 'name':
            sorted_names = sorted(users_in_year)
        else:
            sorted_names = sorted(users_in_year, key=lambda n: year_counts.get(n, 0), reverse=True)

        return [user_checkboxes_dict[name] for name in sorted_names]

    # Initialize with latest year
    initial_year = year_options[0]
    update_checkbox_descriptions(initial_year)
    user_checkboxes = get_sorted_checkboxes('name', initial_year)

    # Styled buttons with custom classes for consistent aesthetic
    select_all_btn = widgets.Button(
        description='Select All',
        button_style='',
        layout=widgets.Layout(width='auto', min_width='100px', height='32px')
    )
    select_all_btn.add_class('wrapped-btn-select')

    deselect_all_btn = widgets.Button(
        description='Clear',
        button_style='',
        layout=widgets.Layout(width='auto', min_width='80px', height='32px')
    )
    deselect_all_btn.add_class('wrapped-btn-clear')

    def select_all(btn):
        # Only select checkboxes that are currently visible (in the year)
        users_in_year = get_users_for_year(year_dropdown.value)
        for name in users_in_year:
            user_checkboxes_dict[name].value = True

    def deselect_all(btn):
        # Only deselect checkboxes that are currently visible (in the year)
        users_in_year = get_users_for_year(year_dropdown.value)
        for name in users_in_year:
            user_checkboxes_dict[name].value = False

    select_all_btn.on_click(select_all)
    deselect_all_btn.on_click(deselect_all)

    # Report name text input (required)
    report_name_input = widgets.Text(
        value='',
        placeholder='Enter a name for your report (required)',
        layout=widgets.Layout(width='auto', min_width='280px', max_width='100%')
    )

    # Generate button with custom class
    generate_button = widgets.Button(
        description='🚀 Generate Report',
        button_style='success',
        layout=widgets.Layout(width='auto', min_width='220px', height='50px')
    )
    generate_button.add_class('wrapped-generate-btn')

    output_area = widgets.Output()
    download_area = widgets.Output()

    # Store file paths for download callbacks
    generated_files = {'html': None, 'static': None}

    # Display styled chat info header (in a widget so we can hide it later)
    format_label = "📱 Static HTML (Mobile) + 💻 Interactive HTML (Desktop)"
    format_badge_color = "#1DB954"

    header_widget = widgets.HTML(f"""
    <div class="wrapped-config-card" style="width: 100%; box-sizing: border-box;">
        <div class="wrapped-header" style="width: 100%; box-sizing: border-box;">
            <h3>📊 {analysis['filename']}</h3>
            <div class="stats">
                <span class="stat-item">💬 {analysis['total_messages']:,} messages</span>
                <span class="stat-item">👥 {analysis['total_members']} members</span>
                <span class="stat-item">📅 Years: {', '.join(str(y) for y in analysis['available_years'])}</span>
            </div>
        </div>
        <div style="display: inline-block; background: {format_badge_color}; color: white; padding: 6px 14px;
                    border-radius: 20px; font-size: 12px; font-weight: 600; margin-bottom: 16px;">
            Output: {format_label}
        </div>
    </div>
    """)

    # Create a scrollable container for checkboxes with styling
    checkbox_container = widgets.VBox(
        user_checkboxes,
        layout=widgets.Layout(
            max_height='220px',
            overflow_y='auto',
            padding='4px',
            width='auto',
            min_width='280px',
            max_width='100%'
        )
    )
    checkbox_container.add_class('wrapped-checkbox-container')

    # Observer to update checkbox list when sort changes
    def on_sort_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            sorted_checkboxes = get_sorted_checkboxes(change['new'], year_dropdown.value)
            checkbox_container.children = sorted_checkboxes

    sort_by_dropdown.observe(on_sort_change, names='value')

    # Year section - using HBox for vertical alignment
    year_section = widgets.HBox([
        widgets.HTML('<div style="color: #1DB954; font-size: 12px; font-weight: 600; text-transform: uppercase; letter-spacing: 0.5px; min-width: 60px;">📅 YEAR</div>'),
        year_dropdown
    ], layout=widgets.Layout(margin='0 0 20px 0', align_items='center', gap='12px'))

    # Member count widget (will be updated dynamically)
    initial_user_count = len(get_users_for_year(initial_year))
    member_count_widget = widgets.HTML(
        f'<div style="color: #888; font-size: 11px; margin-bottom: 8px;">{initial_user_count} member(s) in {initial_year}</div>'
    )

    # Update the year observer to also update the member count
    def on_year_change_extended(change):
        if change['type'] == 'change' and change['name'] == 'value':
            new_year = change['new']
            # Update checkbox descriptions with year-specific counts
            update_checkbox_descriptions(new_year)
            # Update the visible checkboxes list
            sorted_checkboxes = get_sorted_checkboxes(sort_by_dropdown.value, new_year)
            checkbox_container.children = sorted_checkboxes
            # Update member count display
            user_count = len(get_users_for_year(new_year))
            member_count_widget.value = f'<div style="color: #888; font-size: 11px; margin-bottom: 8px;">{user_count} member(s) in {new_year}</div>'

    # Replace the old observer
    year_dropdown.observe(on_year_change_extended, names='value')

    # Users section
    users_section = widgets.VBox([
        widgets.HTML('<div style="color: #1DB954; font-size: 12px; font-weight: 600; margin-bottom: 8px; text-transform: uppercase; letter-spacing: 0.5px;">👥 Members</div>'),
        member_count_widget,
        widgets.HBox([
            select_all_btn,
            deselect_all_btn
        ], layout=widgets.Layout(margin='0 0 8px 0', align_items='center')),
        widgets.HBox([
            widgets.HTML('<span style="color: #888; font-size: 12px; line-height: 32px; margin-right: 8px;">Sort by:</span>'),
            sort_by_dropdown
        ], layout=widgets.Layout(margin='0 0 12px 0', align_items='center')),
        checkbox_container
    ], layout=widgets.Layout(margin='0 0 20px 0'))

    # Report name section (required)
    name_section = widgets.VBox([
        widgets.HTML('<div style="color: #1DB954; font-size: 12px; font-weight: 600; margin-bottom: 8px; text-transform: uppercase; letter-spacing: 0.5px;">📝 Report Name <span style="color: #ff6b6b;">*</span></div>'),
        report_name_input,
        widgets.HTML('<div style="color: #888; font-size: 11px; margin-top: 4px;">This name will be used for the generated report file</div>')
    ], layout=widgets.Layout(margin='0 0 24px 0'))

    # Configuration header
    config_header = widgets.HTML("""
    <div style="color: #1DB954; font-size: 13px; font-weight: 600; text-transform: uppercase;
                letter-spacing: 1px; margin-bottom: 24px; padding-bottom: 12px;
                border-bottom: 1px solid rgba(29, 185, 84, 0.2);">
        ⚙️ Configure your report
    </div>
    """)

    # Store the main input container so we can hide it later
    # Wrap everything in a styled card
    input_container = widgets.VBox([
        config_header,
        year_section,
        users_section,
        name_section,
        generate_button
    ], layout=widgets.Layout(padding='24px'))
    input_container.add_class('wrapped-config-card')

    display(widgets.VBox([
        header_widget,
        input_container,
        output_area,
        download_area
    ]))

    def generate_report(btn):
        # Reset generated files and clear download area on each generation attempt
        generated_files['html'] = None
        generated_files['static'] = None
        with download_area:
            clear_output()

        with output_area:
            clear_output()

            # Validate report name (required)
            custom_name = report_name_input.value.strip()
            if not custom_name:
                display(HTML("""
                <div style="background: linear-gradient(135deg, #d32f2f, #191414); padding: 20px; border-radius: 12px; margin-top: 10px; max-width: 720px; width: 100%; box-sizing: border-box;
                            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                            border: 1px solid rgba(211, 47, 47, 0.15);">
                    <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ Report name required</h4>
                    <p style="color: #ffcdd2; margin: 0;">Please enter a name for your report in the <strong>"Report Name"</strong> field above.</p>
                </div>
                """))
                return

            # Get selected users from checkboxes (extract name from "Name (count)" format)
            selected_users = []
            for name in member_names:
                checkbox = user_checkboxes_dict[name]
                if checkbox.value:
                    selected_users.append(name)

            if not selected_users:
                display(HTML("""
                <div style="background: linear-gradient(135deg, #d32f2f, #191414); padding: 20px; border-radius: 12px; margin-top: 10px; max-width: 720px; width: 100%; box-sizing: border-box;
                            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                            border: 1px solid rgba(211, 47, 47, 0.15);">
                    <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ No users selected</h4>
                    <p style="color: #ffcdd2; margin: 0;">Please select at least one user to include in the report.</p>
                </div>
                """))
                return

            print("\n" + "="*60)
            print("🔄 Generating your WhatsApp Wrapped report...")
            print("="*60 + "\n")

            try:
                # Import modules
                repo_path = "/content/whatsapp-wrapped"
                if repo_path not in sys.path:
                    sys.path.insert(0, repo_path)

                from jinja2 import Environment, FileSystemLoader

                from whatsapp_wrapped.analytics import analyze_chat, format_hour, get_hour_emoji
                from whatsapp_wrapped.charts import (
                    ChartCollection,
                    chart_to_html,
                    create_user_hourly_sparkline,
                    create_user_sparkline,
                )
                from whatsapp_wrapped.parser import parse_whatsapp_export

                # Get widget values
                year_filter = year_dropdown.value
                fixed = True  # Always use fixed layout

                year_value = int(year_filter)  # Always a specific year

                # Parse chat with filters
                import gc

                from whatsapp_wrapped.parser import get_chat_metadata

                def parse_task():
                    df, metadata = parse_whatsapp_export(
                        analysis["chat_path"],
                        filter_system=True,
                        min_messages=1,
                        year_filter=year_value,
                    )
                    df = df.filter(pl.col("name").is_in(selected_users))
                    metadata = get_chat_metadata(df, analysis["filename"])
                    gc.collect()
                    return df, metadata

                df, metadata = run_task_with_spinner(
                    parse_task,
                    f"📖 Parsing chat file (Year: {year_filter}, Users: {len(selected_users)})..."
                )
                print(f"      Found {len(df):,} messages from {metadata.total_members} members")

                # Run analytics
                def analytics_task():
                    return analyze_chat(df)

                analytics = run_task_with_spinner(
                    analytics_task,
                    "📊 Running analytics..."
                )
                print(f"      Analyzed {analytics.total_days} days of chat history")

                # Generate charts
                def charts_task():
                    chart_collection = ChartCollection(analytics)
                    charts_html = chart_collection.to_html_dict(include_plotlyjs_first=True)
                    user_sparklines = {}
                    user_hourly_sparklines = {}
                    for user_stat in analytics.user_stats:
                        sparkline_fig = create_user_sparkline(user_stat.daily_activity, user_stat.name)
                        user_sparklines[user_stat.name] = chart_to_html(sparkline_fig, include_plotlyjs=False)
                        hourly_sparkline_fig = create_user_hourly_sparkline(user_stat.hourly_activity, user_stat.name)
                        user_hourly_sparklines[user_stat.name] = chart_to_html(hourly_sparkline_fig, include_plotlyjs=False)
                    return chart_collection, charts_html, user_sparklines, user_hourly_sparklines

                chart_collection, charts_html, user_sparklines, user_hourly_sparklines = run_task_with_spinner(
                    charts_task,
                    "📈 Generating visualizations..."
                )
                print(f"      Created {len(charts_html)} charts and {len(user_sparklines)} sparklines")

                # Memory cleanup
                del df
                gc.collect()

                # Calculate user badges
                from whatsapp_wrapped.analytics import calculate_badges
                user_badges = calculate_badges(analytics.user_stats)

                # Render HTML
                def render_task():
                    template_dir = Path("/content/whatsapp-wrapped/whatsapp_wrapped/templates")
                    env = Environment(loader=FileSystemLoader(template_dir), autoescape=False)
                    template = env.get_template("report.html")
                    formatted_hour = format_hour(analytics.most_active_hour)
                    hour_emoji = get_hour_emoji(analytics.most_active_hour)
                    return template.render(
                        metadata=metadata,
                        analytics=analytics,
                        charts=charts_html,
                        user_sparklines=user_sparklines,
                        user_hourly_sparklines=user_hourly_sparklines,
                        user_badges=user_badges,
                        generation_date=datetime.now().strftime("%Y-%m-%d %H:%M"),
                        fixed_layout=fixed,
                        formatted_hour=formatted_hour,
                        hour_emoji=hour_emoji,
                        report_title=custom_name,
                    )

                html_content = run_task_with_spinner(
                    render_task,
                    "🎨 Rendering HTML report..."
                )

                # Save report with sanitized, human-friendly filename
                safe_name = sanitize_filename(custom_name)
                output_filename = f"{safe_name} - Wrapped Report.html"

                output_path = f"/content/{output_filename}"
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(html_content)

                # Store HTML path for download
                generated_files['html'] = output_path

                # Memory cleanup: delete large objects after saving
                del html_content, chart_collection, charts_html
                gc.collect()

                # Generate Static HTML for WhatsApp sharing
                static_output_path = None
                static_error_msg = None
                if generate_static_html:
                    from whatsapp_wrapped.generator import (
                        generate_static_html as create_static_html,
                    )
                    static_output_path = f"/content/{safe_name} - Wrapped Report (Static).html"

                    try:
                        def static_task():
                            create_static_html(output_path, static_output_path, quiet=True)
                            return True

                        run_task_with_spinner(
                            static_task,
                            "📱 Converting to Static HTML (for WhatsApp sharing)..."
                        )

                        # Verify the file was actually created before declaring success
                        if os.path.exists(static_output_path):
                            generated_files['static'] = static_output_path
                        else:
                            static_error_msg = "File was not created after conversion"
                            static_output_path = None
                            generated_files['static'] = None
                    except Exception as e:
                        static_error_msg = str(e)
                        import traceback as tb_module
                        print(f"\n  ❌ Static HTML generation failed: {e}")
                        print("     Full traceback:")
                        tb_module.print_exc()
                        static_output_path = None
                        generated_files['static'] = None

                # Memory cleanup: final cleanup before displaying results
                gc.collect()

                # Clear the verbose progress output and show clean success
                clear_output()

                # Hide the header and input container
                header_widget.layout.display = 'none'
                input_container.layout.display = 'none'

                # Display summary
                display(HTML(f"""
                <div style="background: linear-gradient(135deg, #1DB954, #191414);
                            padding: 25px; border-radius: 12px; margin: 20px 0; max-width: 720px; width: 100%; box-sizing: border-box;
                            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);">
                    <h2 style="color: white; margin: 0 0 15px 0;">📊 {metadata.filename}</h2>
                    <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(120px, 1fr)); gap: 15px;">
                        <div style="background: rgba(0,0,0,0.3); padding: 15px; border-radius: 8px; text-align: center;">
                            <div style="color: #1DB954; font-size: 28px; font-weight: bold;">{analytics.total_messages:,}</div>
                            <div style="color: #e8e8e8; font-size: 12px;">MESSAGES</div>
                        </div>
                        <div style="background: rgba(0,0,0,0.3); padding: 15px; border-radius: 8px; text-align: center;">
                            <div style="color: #1DB954; font-size: 28px; font-weight: bold;">{analytics.total_members}</div>
                            <div style="color: #e8e8e8; font-size: 12px;">MEMBERS</div>
                        </div>
                        <div style="background: rgba(0,0,0,0.3); padding: 15px; border-radius: 8px; text-align: center;">
                            <div style="color: #1DB954; font-size: 28px; font-weight: bold;">{analytics.total_days}</div>
                            <div style="color: #e8e8e8; font-size: 12px;">DAYS</div>
                        </div>
                    </div>
                    <div style="margin-top: 15px; padding-top: 15px; border-top: 1px solid rgba(255,255,255,0.1);">
                        <span style="color: #e8e8e8;">Date Range:</span>
                        <span style="color: white;">{metadata.date_range_start.strftime('%b %d, %Y')} - {metadata.date_range_end.strftime('%b %d, %Y')}</span>
                    </div>
                </div>
                """))

                # Build file paths display for manual download fallback
                file_paths_html = f"""
                <div style="background: rgba(29, 185, 84, 0.1); border-left: 4px solid #1DB954; padding: 15px; border-radius: 8px; margin: 15px 0;">
                    <h5 style="color: #1DB954; margin: 0 0 10px 0;">📁 Files saved in Colab:</h5>
                    <div style="font-family: 'Monaco', 'Courier New', monospace; font-size: 13px; color: #e8e8e8;">
                        <div style="margin: 5px 0; background: rgba(0,0,0,0.2); padding: 8px 12px; border-radius: 4px;">
                            💻 <code style="color: #1ed760;">{output_path}</code>
                        </div>
                """
                if static_output_path:
                    file_paths_html += f"""
                        <div style="margin: 5px 0; background: rgba(0,0,0,0.2); padding: 8px 12px; border-radius: 4px;">
                            📱 <code style="color: #ff9800;">{static_output_path}</code>
                        </div>
                """
                file_paths_html += """
                    </div>
                    <p style="color: #b3b3b3; margin: 10px 0 0 0; font-size: 12px;">
                        💡 <strong>If auto-download failed:</strong> Click the 📁 folder icon in Colab's left sidebar,
                        find these files in <code>/content/</code>, right-click and select "Download"
                    </p>
                </div>
                """

                # Build "What's Next?" content based on output format
                if static_output_path:
                    whats_next_content = """
                    <ul style="color: #e8e8e8; margin: 0; padding-left: 20px;">
                        <li>📥 Click the <strong>download buttons below</strong> or use the file paths to download manually</li>
                        <li>📱 <strong>Static HTML (Mobile)</strong> — Share in WhatsApp groups or open on phone</li>
                        <li>💻 <strong>Interactive HTML (Desktop)</strong> — Open in browser for hover tooltips, zoom & pan</li>
                        <li>💡 <strong>Tip:</strong> Files persist in <code>/content/</code> even if the kernel crashes</li>
                        <li>Share the report with your group members!</li>
                        <li>Run this cell again to generate another report with different options!</li>
                    </ul>
                    """
                else:
                    whats_next_content = """
                    <ul style="color: #e8e8e8; margin: 0; padding-left: 20px;">
                        <li>📥 Click the <strong>download button below</strong> or use the file path to download manually</li>
                        <li>Open the downloaded HTML file in any web browser</li>
                        <li>💡 <strong>Tip:</strong> File persists in <code>/content/</code> even if the kernel crashes</li>
                        <li>⚠️ Static HTML generation failed — the interactive version still works great on PC!</li>
                        <li>Run this cell again to try generating both formats</li>
                    </ul>
                    """

                display(HTML(f"""
                {file_paths_html}

                <div style="background: linear-gradient(145deg, #1e1e1e 0%, #2d2d2d 100%); padding: 20px; border-radius: 12px; margin-top: 20px; max-width: 720px; width: 100%; box-sizing: border-box;
                            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                            border: 1px solid rgba(255, 255, 255, 0.05);">
                    <h4 style="color: #1DB954; margin: 0 0 10px 0;">💡 What's Next?</h4>
                    {whats_next_content}
                </div>
                """))

            except Exception as e:
                import traceback
                print(f"\n❌ Error generating report: {e}")
                print("\nFull error details:")
                traceback.print_exc()

        # Create download buttons OUTSIDE the output_area context
        # This ensures files.download() works properly
        with download_area:
            if generated_files['html']:
                # Create download buttons list - Mobile first, then Desktop
                download_buttons = []

                # Static HTML (Mobile) button - shown first when available
                if generated_files['static']:
                    static_download_btn = widgets.Button(
                        description='📱 Download Static HTML (Mobile)',
                        button_style='',
                        layout=widgets.Layout(width='auto', min_width='200px', height='40px')
                    )
                    static_download_btn.add_class('wrapped-btn-download-static')

                    def download_static(btn):
                        files.download(generated_files['static'])
                    static_download_btn.on_click(download_static)
                    download_buttons.append(static_download_btn)

                # Interactive HTML (Desktop) button
                html_download_btn = widgets.Button(
                    description='💻 Download Interactive HTML (Desktop)',
                    button_style='',
                    layout=widgets.Layout(width='auto', min_width='200px', height='40px')
                )
                html_download_btn.add_class('wrapped-btn-download-html')

                def download_html(btn):
                    files.download(generated_files['html'])
                html_download_btn.on_click(download_html)
                download_buttons.append(html_download_btn)

                # Display download section
                display(HTML("""
                <div style="background: linear-gradient(145deg, #1e1e1e 0%, #2d2d2d 100%); padding: 20px; border-radius: 12px; margin-top: 20px; max-width: 720px; width: 100%; box-sizing: border-box;
                            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
                            border: 1px solid rgba(255, 255, 255, 0.05);">
                    <h4 style="color: #1DB954; margin: 0 0 8px 0;">📥 Download Your Reports</h4>
                    <p style="color: #888; margin: 0; font-size: 13px;">Click the buttons below to download your generated reports.</p>
                </div>
                """))
                display(widgets.HBox(download_buttons, layout=widgets.Layout(gap='10px', flex_wrap='wrap')))

    generate_button.on_click(generate_report)

---

<div style="background: linear-gradient(145deg, #1e1e1e 0%, #2d2d2d 100%); padding: 30px; border-radius: 12px; margin: 30px 0; font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif; box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3); border: 1px solid rgba(255, 255, 255, 0.05);">
  
### 🔒 Privacy & Security

<div style="background: rgba(29, 185, 84, 0.1); border-left: 4px solid #1DB954; padding: 15px; border-radius: 8px; margin: 15px 0;">
  
✓ **No third-party servers** - your data stays within Google's infrastructure  
✓ **Ephemeral storage** - your chat data is automatically deleted when the Colab session ends  
✓ **No permanent storage** - nothing is saved to Google Drive or any database  
✓ **Open source & transparent** - audit the code yourself on [GitHub](https://github.com/Duelion/whatsapp-wrapped)

</div>

<div style="background: rgba(255, 193, 7, 0.1); border-left: 4px solid #FFC107; padding: 15px; border-radius: 8px; margin: 15px 0;">
  
⚠️ **Note:** When you upload your chat file, it is temporarily stored on Google's cloud servers (the Colab runtime). 
Google's standard [privacy policies](https://policies.google.com/privacy) apply. 
If privacy is a concern, consider running this project locally instead - see the [GitHub README](https://github.com/Duelion/whatsapp-wrapped) for instructions.

</div>

---

<div style="text-align: center; padding: 20px 0;">
  <h3 style="color: #1DB954; margin-bottom: 10px;">Made with ❤️ for WhatsApp users who love data</h3>
  <p style="color: #e8e8e8; margin-bottom: 25px;">Created by <a href="https://github.com/Duelion" style="color: #1DB954; text-decoration: none;">@Duelion</a></p>
  
  <p>
    <a href="https://buymeacoffee.com/duelion" target="_blank" rel="noopener noreferrer"><img src="https://cdn.buymeacoffee.com/buttons/v2/default-red.png" alt="Buy Me A Coffee" width="150"></a>
  </p>
  <p>
    <a href="https://github.com/Duelion/whatsapp-wrapped" target="_blank" rel="noopener noreferrer"><img src="https://img.shields.io/github/stars/Duelion/whatsapp-wrapped?style=for-the-badge&logo=github" alt="GitHub Stars"></a>
  </p>
  
  <p style="color: #666; margin-top: 25px; font-size: 12px;">
    💡 <strong>Enjoying WhatsApp Wrapped?</strong> Share it with your friends and star the repo!
  </p>
</div>

</div>